<a href="https://colab.research.google.com/github/petalzx/amazonReviewScraper/blob/main/amazonReviewScaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amazon Review Web Scaper

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
#Modify the search_query in order to create data set from a different search
search_query = 'headphones'
base_url = 'https://www.amazon.com/s?k='

In [3]:
url = base_url + search_query
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 
          'referer':'https://www.amazon.com/s?k=headphones&crid=3RIA528VORR9E&sprefix=headphones%2Caps%2C107&ref=nb_sb_noss_2'}

In [4]:
#generating search response
response = requests.get(url, headers=header)
response.status_code

200

In [5]:
#scape Amazon reviews of products that pop up from a specific search_query
def search(search_query):
  url = 'https://www.amazon.com/s?k=' + search_query
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [6]:
#scape Amazon reviews of individual product pages using asin indentification number
def asinSearch(asin):
  url = 'https://www.amazon.com/dp/' + asin
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [7]:
#brings us to "see all reviews" page
def reviewSearch(review_url):
  url = "https://www.amazon.com" + review_url
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [8]:
product_names = []
response = search(search_query)
soup = bs(response.content)
for i in soup.find_all('span', {'class':'a-size-medium a-color-base a-text-normal'}):
  product_names.append(i.text)

https://www.amazon.com/s?k=headphones


In [9]:
product_price = []
for i in soup.find_all('span', {'class':'a-offscreen'}):
  product_price.append(i.text)

In [10]:
product_asin = []
for i in soup.find_all('div',{'class':'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16'}):
  product_asin.append(i['data-asin'])

In [11]:
product_review_url = []
for i in range(len(product_asin)):
  response = asinSearch(product_asin[i])
  soup = bs(response.content)
  product_review_url.append(soup.find('a',{'class':'a-link-emphasis a-text-bold'}).get('href'))

https://www.amazon.com/dp/B08528YFM3
https://www.amazon.com/dp/B07YVZ15SK
https://www.amazon.com/dp/B01M0GB8CC
https://www.amazon.com/dp/B08WM3LMJF
https://www.amazon.com/dp/B08VNFD8FS
https://www.amazon.com/dp/B09WQ1FB5R
https://www.amazon.com/dp/B0BLZ9PJVD
https://www.amazon.com/dp/B00NJ2M33I
https://www.amazon.com/dp/B08WR6YDKP
https://www.amazon.com/dp/B06X16Z7DZ
https://www.amazon.com/dp/B094NC89P9
https://www.amazon.com/dp/B07NM3RSRQ
https://www.amazon.com/dp/B08G1XKY6N
https://www.amazon.com/dp/B08FKB4476
https://www.amazon.com/dp/B09KGLRF8J
https://www.amazon.com/dp/B095J62YQR


In [12]:
#All of the data categories listed out
id_data = []
name_data = []
price_data = []
asin_data = []
#reviewer_name_data = []
title_data = []
text_data = []
score_data = []

In [13]:
#collects data for all categories, loops through each product from the search and collects data from reviews of that product
def productScrape():
  #id_number = 0
  for i in range(len(product_review_url)):
    count = 0
    
    response = reviewSearch(product_review_url[i])
    soup = bs(response.content)

    score_raw = soup.find_all('i', class_=lambda x: x and 'review-rating' in x.split() and 'cr-lightbox-review-rating' not in x.split())
    title_raw = soup.find_all('a', {'data-hook':'review-title'})

    for j in soup.find_all('span',{'data-hook':'review-body'}):
      #id_data.append(id_number)
      #id_number+=1

      name_data.append(product_names[i])
      price_data.append(product_price[i])
      asin_data.append(product_asin[i])

      text_data.append(j.text)

      score_data.append(score_raw[count+2].text[:1])
      title_data.append(title_raw[count].text)
      count+=1

In [14]:
productScrape()
data = {'Name':name_data, 'Price':price_data, 'Asin #':asin_data, #'Reviewer Name':reviewer_name_data,
        'Title':title_data, 'Text':text_data, 'Score':score_data}
df = pd.DataFrame(data)

https://www.amazon.com/Beats-Studio3-Wireless-Headphones-Collection/product-reviews/B08528YFM3/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Beats-Solo3-Wireless-Ear-Headphones/product-reviews/B07YVZ15SK/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Apple-EarPods-Lightning-Connector-White/product-reviews/B01M0GB8CC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/JBL-Tune-510BT-Ear-Headphones/product-reviews/B08WM3LMJF/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Wireless-Bluetooth-Headphones-Foldable-Headsets/product-reviews/B08VNFD8FS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Beats-Studio-Cancelling-Earbuds-Built-Bluetooth-Headphones/product-reviews/B09WQ1FB5R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Bluetooth-Headphones-Waterproof-Earphones-Microphone/

In [15]:
df.head(20)

,Name,Price,Asin #,Title,Text,Score
0,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nPlease Read Before Buying.\n,\nMy rating falls close to a 1.5 for multiple ...,2
1,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,"\nReally Beautiful and Quality Headphones, Bew...",\nI really like these over the head headphones...,4
2,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nGood but could be better\n,\nI got these headphones to wear them when I w...,4
3,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nAMAZINING headphones better than the Solo 3 ...,\nAMAZING headphones that perform better than ...,5
4,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nPerfect headphones for traveling and any noi...,\nThese are the perfect noise-canceling headph...,5
5,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,"\nThey’re great, but I have a few words of adv...","\n1.) There quality and volume is amazing, but...",4
6,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nAmazing\n,"\nthe box came it new. came with the headset, ...",5
7,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nLove Love Love\n,\nI purchased these because I had the wired ve...,5
8,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nLong battery life and amazing headphones\n,\nI love these headphones. The battery life is...,5
9,Beats Studio3 Wireless Noise Cancelling Over-E...,$349.95,B08528YFM3,\nNice product\n,"\nThese headphones are very comfortable, sound...",5


In [16]:
df.sample(5)

,Name,Price,Asin #,Title,Text,Score
108,Apple EarPods Headphones with 3.5mm Plug. Micr...,$21.99,B094NC89P9,\nPretty decent. Lasted 8 months.\n,\nUsed these to sleep with while listening to ...,5
126,Soundcore Anker Life Q20 Hybrid Active Noise C...,$29.99,B08G1XKY6N,\nI prefer the older version\n,\nThese are a downgrade from SkullCandy Venues...,4
158,"Boean Bluetooth Headphones, Running Wireless E...",$12.99,B095J62YQR,\nAwesome!\n,\nI am really surprised at how amazing these e...,5
91,JBL Live 460NC - Wireless On-Ear Noise Cancell...,$18.69,B06X16Z7DZ,\nFast delivery\n,\nI wanted a pair of these headphones because ...,5
72,Bluetooth Headphones V5.3 Wireless Earbuds 50 ...,$29.95,B00NJ2M33I,\nThe only headphones that don't hurt my ears\n,\nI've tried many headphones. These are the on...,5


In [17]:
#Saves the dataframe as a csv file
df.to_csv('amazon_data.csv', index=True)

In [18]:
#Downloads csv file to local machine
from google.colab import files
files.download('amazon_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>